In [6]:
# coding=utf-8

import re

# !/usr/bin/python3

# !/usr/bin/python3
# CSE354 Sp20; Assignment 1 Template v1
##################################################################

import sys

##################################################################
# 1. Tokenizer

import re  # python's regular expression package


def tokenize(sent):
    # input: a single sentence as a string.
    # output: a list of each "word" in the text
    # must use regular expressions
    # <FILL IN>
    tokens = []
    split = re.split(r' ', sent)
    for word in split:
        if not word[len(word) - 1].isalpha():
            # is abbreviation
            if word[len(word) - 2].isupper():
                tokens.append(word)
            else:
                index = 0
                alphas = ""
                while word[index].isalpha():
                    alphas += word[index]
                    index += 1
                tokens.append(alphas)
                for index in range(index, len(word)):
                    tokens.append(word[index])
                    index += 1

        else:
            tokens.append(word)
    # <FILL IN>

    return tokens



##################################################################
# 2. Pig Latinizer

def pigLatinizer(tokens):
    # input: tokens: a list of tokens,
    # output: plTokens: tokens after transforming to pig latin
    plTokens = []
    for word in tokens:
        almun=word.isalpha()
        if not word.isalpha():
            plTokens.append(word)
        elif word[0].islower():
            if word[0].lower() in "aeiou":
                plTokens.append(word + "way")
            else:
                index = 1
                char = word[index].lower()
                build=word[0]
                while char not in "aeiou" and index < len(word) - 1:
                    build += char
                    index += 1
                    char = word[index].lower()
                plTokens.append(word[index:] + build + "ay")
        else:
            plTokens.append(word+"ay")

    return plTokens

##################################################################
# 3. Feature Extractor

import numpy as np


def getFeaturesForTokens(tokens, wordToIndex):
    # input: tokens: a list of tokens,
    # wordToIndex: dict mapping 'word' to an index in the feature list.
    # output: list of lists (or np.array) of k feature values for the given target

    num_words = len(tokens)
    featuresPerTarget = list()  # holds arrays of feature per word
    for targetI in range(num_words):
        index=wordToIndex(tokens[targetI]);
        # <FILL IN>
        featuresPerTarget[index] = [0]*((3*num_words)+2)
        next_word , prev_word=None , None
        vowels, consonants = 0, 0
        word=tokens[targetI]
        featuresPerTarget[index][targetI+num_words+2]=1
        if targetI > 0:
            featuresPerTarget[targetI][targetI-1]=1
        if targetI < num_words-1:
            next_word= tokens[targetI+1]
            featuresPerTarget[index][2+targetI+1+(2*num_words)]=1
        for char in word:
            if char in "aeiou":
                vowels += 1
            else:
                consonants +=1
        featuresPerTarget[index][0]=vowels
        featuresPerTarget[index][1]=consonants
        
        pass
    
        # 

    return featuresPerTarget  # a (num_words x k) matrix


##################################################################
# 4. Adjective Classifier

from sklearn.linear_model import LogisticRegression

                                                  
def trainAdjectiveClassifier(features, adjs):
    # inputs: features: feature vectors (i.e. X)
    #        adjs: whether adjective or not: [0, 1] (i.e. y)
    # output: model -- a trained sklearn.linear_model.LogisticRegression object

    model = None
    # <FILL IN>

    return model


##################################################################
##################################################################
## Main and provided complete methods
## Do not edit.
## If necessary, write your own main, but then make sure to replace
## and test with this before you submit.
##
## Note: Tests below will be a subset of those used to test your
##       code for grading.

def getConllTags(filename):
    # input: filename for a conll style parts of speech tagged file
    # output: a list of list of tuples
    #        representing [[[word1, tag1], [word2, tag2]]]
    wordTagsPerSent = [[]]
    sentNum = 0
    with open(filename, encoding='utf8') as f:
        for wordtag in f:
            wordtag = wordtag.strip()
            if wordtag:  # still reading current sentence
                (word, tag) = wordtag.split("\t")
                wordTagsPerSent[sentNum].append((word, tag))
            else:  # new sentence
                wordTagsPerSent.append([])
                sentNum += 1
    return wordTagsPerSent


# Main
if __name__ == '__main__':
    # Data for 1 and 2
    testSents = ['I am attending NLP class 2 days a week at S.B.U. this Spring.',
                 "I don't think data-driven computational linguistics is very tough.",
                 '@mybuddy and the drill begins again. #SemStart']

    # 1. Test Tokenizer:
    print("\n[ Tokenizer Test ]\n")
    tokenizedSents = []
    for s in testSents:
        tokenizedS = tokenize(s)
        print(s, tokenizedS, "\n")
        tokenizedSents.append(tokenizedS)

    # 2. Test Pig Latinizer:
    print("\n[ Pig Latin Test ]\n")
    for ts in tokenizedSents:
        print(ts, pigLatinizer(ts), "\n")

    # load data for 3 and 4 the adjective classifier data:
    taggedSents = getConllTags('daily547.conll')

    # 3. Test Feature Extraction:
    print("\n[ Feature Extraction Test ]\n")
    # first make word to index mapping:
    wordToIndex = set()  # maps words to an index
    for sent in taggedSents:
        if sent:
            words, tags = zip(*sent)  # splits [(w, t), (w, t)] into [w, w], [t, t]
            wordToIndex |= set(words)  # union of the words into the set
    print("  [Read ", len(taggedSents), " Sentences]")
    # turn set into dictionary: word: index
    wordToIndex = {w: i for i, w in enumerate(wordToIndex)}

    # Next, call Feature extraction per sentence
    sentXs = []
    sentYs = []
    print("  [Extracting Features]")
    for sent in taggedSents:
        if sent:
            words, tags = zip(*sent)
            sentXs.append(getFeaturesForTokens(words, wordToIndex))
            sentYs.append([1 if t == 'A' else 0 for t in tags])
    # test sentences
    print("\n", taggedSents[5], "\n", sentXs[5], "\n")
    print(taggedSents[192], "\n", sentXs[192], "\n")

    # 4. Test Classifier Model Building
    print("\n[ Feature Extraction Test ]\n")
    # setup train/test:
    from sklearn.model_selection import train_test_split

    # flatten by word rather than sent:
    X = [j for i in sentXs for j in i]
    y = [j for i in sentYs for j in i]
    try:
        X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                            test_size=0.20,
                                                            random_state=42)
    except ValueError:
        print("\nLooks like you haven't implemented feature extraction yet.")
        print("[Ending test early]")
        sys.exit(1)
    print("  [Broke into training/test. X_train is ", X_train.shape, "]")
    # Train the model.
    print("  [Training the model]")
    tagger = trainAdjectiveClassifier(X_train, y_train)
    print("  [Done]")

    # Test the tagger.
    from sklearn.metrics import classification_report

    # get predictions:
    y_pred = tagger.predict(X_test)
    # compute accuracy:
    leny = len(y_test)
    print("test n: ", leny)
    acc = np.sum([1 if (y_pred[i] == y_test[i]) else 0 for i in range(leny)]) / leny
    print("Accuracy: %.4f" % acc)
    print(classification_report(y_test, y_pred, ['not_adj', 'adjective']))



                                                  
                                                  




[ Tokenizer Test ]

I am attending NLP class 2 days a week at S.B.U. this Spring. ['I', 'am', 'attending', 'NLP', 'class', '', '2', 'days', 'a', 'week', 'at', 'S.B.U.', 'this', 'Spring', '.'] 

I don't think data-driven computational linguistics is very tough. ['I', "don't", 'think', 'data-driven', 'computational', 'linguistics', 'is', 'very', 'tough', '.'] 

@mybuddy and the drill begins again. #SemStart ['@mybuddy', 'and', 'the', 'drill', 'begins', 'again', '.', '#SemStart'] 


[ Pig Latin Test ]

['I', 'am', 'attending', 'NLP', 'class', '', '2', 'days', 'a', 'week', 'at', 'S.B.U.', 'this', 'Spring', '.'] ['Iay', 'amway', 'attendingway', 'NLPay', 'assclay', '', '2', 'aysday', 'away', 'eekway', 'atway', 'S.B.U.', 'isthay', 'Springay', '.'] 

['I', "don't", 'think', 'data-driven', 'computational', 'linguistics', 'is', 'very', 'tough', '.'] ['Iay', "don't", 'inkthay', 'data-driven', 'omputationalcay', 'inguisticslay', 'isway', 'eryvay', 'oughtay', '.'] 

['@mybuddy', 'and', 'the', 'dri

FileNotFoundError: [Errno 2] No such file or directory: 'daily547.conll'